In this code I will create a class game and player that play a game and store it's possible outcomes.

In [1]:
def getFactors(i):
    factors=[]
    for j in range(1,i,1):
        if i%j==0:
            factors.append(j)
    return factors

In [2]:
class Player:
    def __init__(self, name):
        self.name=name
        
    def play(self,game,stage,t): 
        arr=stage.array
        n=len(stage.array)
        
        test=arr==game.end # test if given stage is terminal
        op=game.players[t%2] # if turn is odd, the other player is players[1]=2
        
        if test: # if yes, return False
            stage.addTurn(-1)    
            stage.addWinner(op)
            return False
        
        else: # otherwise, make a list of all possible arrays that can be reached. 
              # if these are not already stages make stages
            stage.addTurn(t)
            for i in range(n):
                h=arr[i]
                if h>1:
                    for f in getFactors(h):
                        n_a=arr[:i]+(f,)+arr[i+1:]
                        # if n_a is not in keys of game's stages, create a new stage.
                        if n_a not in game.stages.keys():
                            newStage=Stage(n_a,stage,op)
                        else:
                            newStage=game.stages[(n_a, op.name)]
                        game.addStage(newStage,stage,self)
                        stage.addNext(newStage)                   
            return stage.next

In [3]:
class Stage:
    def __init__(self,arr,ps,p):
        self.id=(arr, p.name)
        self.array=arr
        self.prev=[ps]
        self.next=[]
        self.player=p
        self.turn=None
        self.winner=None
        self.feasible=None
        
    def addPrev(self,ps):
        self.prev.append(ps)
    
    def addNext(self,ns):
        self.next.append(ns)
    
    def addTurn(self,t):
        self.turn=t
    
    def addPlayer(self,p):
        self.player=p
    
    def addWinner(self,wp):
        self.winner=wp

In [6]:
class Game:
    def __init__(self,arr,end_arr,n_p=2):
        self.start=arr
        self.end=end_arr
        self.n_players=n_p
        self.players=[]
        for i in range(1,n_p+1,1):
            newPlayer=Player(i)
            self.players.append(newPlayer)
        init_stage=Stage(arr,None,self.players[0])
        self.stages={}
        self.stages[(arr, self.players[0].name)]=init_stage
        self.game_winner=None
        
    def addStage(self,newStage,prevStage,p):
        if newStage in self.stages:
            if prevStage not in newStage.prev:
                newStage.addPrev(prevStage)        
        else: 
            self.stages[(newStage.array, p.name)]=newStage
                
    def __iter__(self):
        return iter(self.stages.values())
    
    def remStage(self,stage):
        if stage.id in self.stages:
            del self.stages[stage.id]
    
    def remBranch(self,stage): # this method removes all follwing stages, not itself.
        if stage.next:
            for ns in stage.next:
                if ns:
                    self.remBranch(ns)
                    self.remStage(ns)
            #self.remStage(stage) # uncommenting this line will delete the stage as well.
            
    def bi(self,stage):
        if stage.winner:
            w=stage.winner
            if stage.prev:
                for ps in stage.prev:
                    if ps:
                        ps.winner=w
                        if ps.player!=w:
                            self.remBranch(ps)
                            self.bi(ps)
                
                    
    def playGame(self,max_turns):
        player_1=self.players[0]
        player_2=self.players[1]
        turns=0
        next_lstage=[self.stages[(self.start, player_1.name)]]
                     
        # go through all possible moves and make stage out of them
        while turns<max_turns and len(next_lstage)>0:
            turns+=1
            if turns%2==0:
                curr_player=player_1
            else:
                curr_player=player_2
            curr_lstage=next_lstage
            next_lstage=[]
            for stage in curr_lstage:
                n_s=curr_player.play(self,stage,turns)
                if n_s:
                    next_lstage+=n_s
        
        counter=0
        for s in self:
            counter+=1
        print(counter)
        
        # add backward induction here
        # start from turn == -1, and add winner to the stages in prev stages.
        # keep only the stages in which the player is winning at the end.
        
        l=[i for i in self.stages.keys()]
        for k in l:
            if k[0]==self.end:
                if k in self.stages:
                    self.bi(self.stages[k])

In [7]:
tb2=Game((10, 10),(1, 1),2)
tb2.playGame(100) # makes all stages

counter=0
for s in tb2:
    counter+=1
    print(s.id)
print('No. stages={}'.format(counter))

23
((10, 10), 1)
((1, 10), 2)
((2, 10), 2)
((5, 10), 2)
((10, 1), 2)
((10, 2), 2)
((10, 5), 2)
((1, 1), 1)
((1, 2), 1)
((1, 5), 1)
((1, 10), 1)
((2, 1), 1)
((2, 2), 1)
((2, 5), 1)
((5, 1), 1)
((5, 2), 1)
((5, 5), 1)
((10, 1), 1)
((1, 1), 2)
((1, 2), 2)
((1, 5), 2)
((2, 1), 2)
((5, 1), 2)
No. stages=23
